In [3]:
import pandas as pd

dados = pd.read_csv("drive/MyDrive/creditcard.csv")
dados.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
dados.isna().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [5]:
n_transacoes = dados['Class'].count()
n_fraudes = dados['Class'].sum()
n_normais = n_transacoes - n_fraudes
fraudes_porcentagem = n_fraudes / n_transacoes
normais_porcentagem = n_normais / n_transacoes

print("Number of transactions", n_transacoes)
print("Number of frauds %d, com %.2f%%" % (n_fraudes, fraudes_porcentagem*100))
print("Number of normal transactions %d, com %.2f%%" % (n_normais, normais_porcentagem*100))

Number of transactions 284807
Number of frauds 492, com 0.17%
Number of normal transactions 284315, com 99.83%


In [6]:
X = dados.drop('Class', axis=1).values
Y = dados['Class'].values

In [7]:
import matplotlib.pyplot as plt

def save_tree(classifier, name):
  plt.figure(figsize=(200, 100))
  tree.plot_tree(classifier, filled=True, fontsize=14)
  plt.savefig(name)
  plt.close()

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit

def split_traininig_data(X, Y):
  splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
  for treino_id, teste_id in splitter.split(X, Y):
    X_train, X_test = X[treino_id], X[teste_id]
    Y_train, Y_test = Y[treino_id], Y[teste_id]
  return X_train, X_test, Y_train, Y_test

In [9]:
%%time
from sklearn import tree

X_train, X_test, Y_train, Y_test = split_traininig_data(X, Y)

def classify(classifier, X_train, X_test, Y_train):
  decision_tree = classifier.fit(X_train, Y_train)
  y_predict = decision_tree.predict(X_test)
  return y_predict

classifier = tree.DecisionTreeClassifier()
y_predict = classify(classifier, X_train, X_test, Y_train)

CPU times: user 33.8 s, sys: 89.9 ms, total: 33.9 s
Wall time: 37.4 s


In [10]:
save_tree(classifier, "decision_tree.png")

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def validate_classification(Y_test, y_predict):
  print("Accuracy:", accuracy_score(Y_test, y_predict))
  print("Precision Score:", precision_score(Y_test, y_predict))
  print("Recall Score:", recall_score(Y_test, y_predict))
  print("Confusion Matrix:\n", confusion_matrix(Y_test, y_predict))

validate_classification(Y_test, y_predict)

Accuracy: 0.9990168884519505
Precision Score: 0.7058823529411765
Recall Score: 0.7346938775510204
Confusion Matrix:
 [[28417    15]
 [   13    36]]


In [12]:
print(classifier)
print(classifier.get_depth())

DecisionTreeClassifier()
21


In [13]:
%%time
classifier_decision_tree = tree.DecisionTreeClassifier(max_depth=10, random_state=0)
y_predict_decision_tree = classify(classifier_decision_tree, X_train, X_test, Y_train)
validate_classification(Y_test, y_predict_decision_tree)

Accuracy: 0.9994733330992591
Precision Score: 0.9473684210526315
Recall Score: 0.7346938775510204
Confusion Matrix:
 [[28430     2]
 [   13    36]]
CPU times: user 19.8 s, sys: 31.3 ms, total: 19.8 s
Wall time: 20.2 s


In [14]:
classifier = tree.DecisionTreeClassifier(max_depth=10, random_state=0, min_samples_leaf=10)
y_predict = classify(classifier, X_train, X_test, Y_train)
validate_classification(Y_test, y_predict)

Accuracy: 0.9993679997191109
Precision Score: 0.8604651162790697
Recall Score: 0.7551020408163265
Confusion Matrix:
 [[28426     6]
 [   12    37]]


In [15]:
classifier = tree.DecisionTreeClassifier(max_depth=5, random_state=0)
y_predict = classify(classifier, X_train, X_test, Y_train)
validate_classification(Y_test, y_predict)

Accuracy: 0.999403110845827
Precision Score: 0.9210526315789473
Recall Score: 0.7142857142857143
Confusion Matrix:
 [[28429     3]
 [   14    35]]


In [16]:
%%time
from sklearn.ensemble import RandomForestClassifier

classifier_random_forest = RandomForestClassifier(n_estimators=100, random_state=0)
y_predict_random_forest = classify(classifier_random_forest, X_train, X_test, Y_train)
validate_classification(Y_test, y_predict_random_forest)

Accuracy: 0.9995084442259752
Precision Score: 0.9487179487179487
Recall Score: 0.7551020408163265
Confusion Matrix:
 [[28430     2]
 [   12    37]]
CPU times: user 5min 50s, sys: 389 ms, total: 5min 51s
Wall time: 6min


In [17]:
print("Number of trees:", len(classifier_random_forest.estimators_))
save_tree(classifier_random_forest.estimators_[0], "random_forest_1")
save_tree(classifier_random_forest.estimators_[1], "random_forest_2")

Number of trees: 100


In [18]:
%%time
classifier_random_forest = RandomForestClassifier(n_estimators=50, random_state=0, max_depth=10)
y_predict_random_forest = classify(classifier_random_forest, X_train, X_test, Y_train)
validate_classification(Y_test, y_predict_random_forest)

Accuracy: 0.9995435553526912
Precision Score: 0.9736842105263158
Recall Score: 0.7551020408163265
Confusion Matrix:
 [[28431     1]
 [   12    37]]
CPU times: user 1min 40s, sys: 149 ms, total: 1min 41s
Wall time: 1min 40s


In [19]:
print("Number of trees:", len(classifier_random_forest.estimators_))
save_tree(classifier_random_forest.estimators_[0], "random_forest_max_depth_10_1")
save_tree(classifier_random_forest.estimators_[1], "random_forest_max_depth_10_2")

Number of trees: 50


In [22]:
%%time
from sklearn.ensemble import AdaBoostClassifier

adaboost_classifier = AdaBoostClassifier(random_state=0)
y_predict_adaboost = classify(adaboost_classifier, X_train, X_test, Y_train)
validate_classification(Y_test, y_predict_adaboost)

Accuracy: 0.9992626663389628
Precision Score: 0.8888888888888888
Recall Score: 0.6530612244897959
Confusion Matrix:
 [[28428     4]
 [   17    32]]
CPU times: user 1min 41s, sys: 165 ms, total: 1min 42s
Wall time: 1min 42s


In [23]:
print("Number of trees:", len(adaboost_classifier.estimators_))
save_tree(adaboost_classifier.estimators_[0], "ada_boost_1")
save_tree(adaboost_classifier.estimators_[1], "ada_boost_2")

Number of trees: 50


In [24]:
%%time
adaboost_classifier = AdaBoostClassifier(random_state=0, n_estimators=100)
y_predict_adaboost = classify(adaboost_classifier, X_train, X_test, Y_train)
validate_classification(Y_test, y_predict_adaboost)

Accuracy: 0.999403110845827
Precision Score: 0.8636363636363636
Recall Score: 0.7755102040816326
Confusion Matrix:
 [[28426     6]
 [   11    38]]
CPU times: user 3min 23s, sys: 292 ms, total: 3min 24s
Wall time: 3min 23s


In [25]:
%%time
adaboost_classifier = AdaBoostClassifier(random_state=0, n_estimators=200)
y_predict_adaboost = classify(adaboost_classifier, X_train, X_test, Y_train)
validate_classification(Y_test, y_predict_adaboost)

Accuracy: 0.9995435553526912
Precision Score: 0.9285714285714286
Recall Score: 0.7959183673469388
Confusion Matrix:
 [[28429     3]
 [   10    39]]
CPU times: user 6min 47s, sys: 558 ms, total: 6min 47s
Wall time: 6min 47s
